In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import re

df = pd.read_csv("../data/fianl_data_without_entity.csv", encoding = "euc-kr")

In [53]:
df.isnull().sum().sort_values().head(20)

종목코드            0
회사명             0
시장구분            0
업종              0
업종명             0
결산월             0
결산기준일           0
보고서종류           0
자본과부채총계        23
부채총계           24
유동자산           28
비유동자산          28
유동부채           29
자산총계           30
현금및현금성자산       32
유형자산           44
자본총계           64
비유동부채          74
이익잉여금(결손금)    324
자본금           434
dtype: int64

In [54]:
", ".join(df.isnull().sum().sort_values().tail(20).index)

'교환사채상환할증금, 전환상환우선주부채할증발행차금, 장기구조조정충당부채, 신주인수권부사채할증발행차금, 장기손실부담계약충당부채, 전환사채할증발행차금, 사채할증발행차금, 장기공사선수금, 자본 [abstract], 단기공사유보금, 단기구조조정충당부채, 투자선수금, 교환권대가, 장기이연대출부대손익, 비유동 단기매매정의 충족 금융부채, 부채 [abstract], 장기공사미수금, 장기분양미수금, 장기공사유보금, 자산 [abstract]'

In [55]:
df.shape

(11674, 527)

In [56]:
pre1 = df[["회사명", "자본과부채총계"]]
pre1[pre1["자본과부채총계"].isnull()].sort_values("회사명")

회사명  자본과부채총계
3427   HDC현대산업개발      NaN
2089       두산솔루스      NaN
3781       두산솔루스      NaN
445         레드로버      NaN
2137        레드로버      NaN
3829        레드로버      NaN
4126      스타모빌리티      NaN
4157      신한알파리츠      NaN
9121          썬텍      NaN
8028         코스온      NaN
6399         코스온      NaN
4748         코스온      NaN
3056         코스온      NaN
1364         코스온      NaN
9687         코스온      NaN
11356        코스온      NaN
4871          퓨전      NaN
4883       피에스케이      NaN
3191       피에스케이      NaN
5053       효성중공업      NaN
5054      효성첨단소재      NaN
5055       효성티앤씨      NaN
5056        효성화학      NaN

In [68]:
# 자본과부채총계 결측치 처리 Let's get it
# HDC현대산업개발 : 2018 상장
df[df['회사명'] == "HDC현대산업개발"]

# 두산솔루스 : 2019 상장
df[df['회사명'] == "두산솔루스"]

# 레드로버 : 사업보고서에만 자본과부채총계 부재... 좀 더 살펴봐야 함
df[df['회사명'] == "레드로버"]

# 스타모빌리티 : 2018년 이전 데이터 보고 X
df[df['회사명'] == "스타모빌리티"]

# 신한알파리츠 : 2018 사장
df[df['회사명'] == "신한알파리츠"]

# 썬텍 : 상장폐지
df[df['회사명'] == "썬텍"]

# 전처리가 잘못된거 같음......
df[df['회사명'] == "코스온"]

종목코드  회사명       시장구분   업종          업종명  결산월       결산기준일   보고서종류  \
1364   [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12  2019.12.31   사업보고서   
3056   [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12  2018.12.31   사업보고서   
4748   [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12  2017.12.31   사업보고서   
6399   [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   2019.6.30   반기보고서   
8028   [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   2019.3.31  1분기보고서   
9687   [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   2019.9.30  3분기보고서   
11356  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   2020.3.31  1분기보고서   

       재무상태표 [abstract]  자산 [abstract]  ...  현금흐름위험회피파생상품평가손실  지분법기타포괄손익변동  \
1364                NaN            NaN  ...               NaN          NaN   
3056                NaN            NaN  ...               NaN          NaN   
4748                NaN            NaN  ...               NaN          NaN   
6399                NaN            NaN  ...               NaN          NaN   
8028                NaN            NaN  ...               NaN          NaN   
9687                NaN            NaN  ...               NaN          NaN   
11356               NaN            NaN  ...               NaN          NaN   

         이익잉여금(결손금)  이익준비금  기타법정적립금  임의적립금  미처분이익잉여금(미처리결손금)         비지배지분  \
1364   2.375868e+09    NaN      NaN    NaN      2.375868e+09  2.107977e+09   
3056   7.687881e+09    NaN      NaN    NaN      7.687881e+09  3.495514e+09   
4748   1.066239e+10    NaN      NaN    NaN      1.066239e+10  1.391041e+09   
6399   6.042439e+09    NaN      NaN    NaN      6.042439e+09  3.595453e+09   
8028   7.811125e+09    NaN      NaN    NaN      7.811125e+09  3.757284e+09   
9687   5.032832e+09    NaN      NaN    NaN      5.032832e+09  3.363337e+09   
11356 -2.143817e+09    NaN      NaN    NaN     -2.143817e+09  1.985228e+09   

       자본총계  자본과부채총계  
1364    NaN      NaN  
3056    NaN      NaN  
4748    NaN      NaN  
6399    NaN      NaN  
8028    NaN      NaN  
9687    NaN      NaN  
11356   NaN      NaN  

[7 rows x 527 columns]

In [61]:
df.iloc[4126,:].isnull()

519

In [ ]:
df = df.drop(3427).reset_index(drop=True)

In [63]:
a = pd.read_csv("../data/2017_사업보고서_01_재무상태표_연결_20200626.txt",  sep = "\t", encoding = "cp949")

In [69]:
a[a["회사명"] == "코스온"]

재무제표종류      종목코드  회사명       시장구분   업종          업종명  결산월  \
59056  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59057  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59058  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59059  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59060  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59061  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59062  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59063  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59064  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59065  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59066  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59067  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59068  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59069  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59070  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59071  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59072  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59073  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59074  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59075  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59076  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59077  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59078  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59079  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59080  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59081  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59082  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59083  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59084  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59085  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59086  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59087  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59088  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59089  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59090  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59091  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59092  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59093  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59094  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59095  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59096  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59097  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59098  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59099  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59100  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59101  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59102  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59103  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법인  204  기타 화학제품 제조업   12   
59104  재무상태표, 유동/비유동법-연결재무제표  [069110]  코스온  코스닥시장상장법